In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier

# 1. 데이터 불러오기
df = pd.read_csv('/content/XGBoost_features.csv')  # 파일명/경로를 맞춰주세요

# 2. 특성(features)과 레이블(label) 분리
#    컬럼 인덱스로 지정: iloc[:, 3:] → 4번째 컬럼(인덱스 3)부터 끝까지가 피처
X = df.iloc[:, 3:]
y = df.iloc[:, 2]

# 3. 학습용/검증용 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. XGBoost 분류기 생성 및 학습
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)
model.fit(X_train, y_train)

# 5. 예측 및 평가
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 6. (선택) 모델 저장
import joblib
joblib.dump(model, 'xgb_fall_detector.pkl')


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:20:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.8182679552776199

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.81      0.82      2726
           1       0.80      0.83      0.82      2551

    accuracy                           0.82      5277
   macro avg       0.82      0.82      0.82      5277
weighted avg       0.82      0.82      0.82      5277



In [ ]:
# ——————————————————————————————
# 전체 레이블 분포 확인
total_counts = y.value_counts()
print("전체 데이터 분포")
print(total_counts)
print("전체 비율")
print((total_counts / len(y)).round(3), "\n")

# 학습/검증 레이블 분포 확인
train_counts = y_train.value_counts()
test_counts  = y_test.value_counts()

print("학습 세트 분포")
print(train_counts)
print("학습 세트 비율")
print((train_counts / len(y_train)).round(3), "\n")

print("테스트 세트 분포")
print(test_counts)
print("테스트 세트 비율")
print((test_counts / len(y_test)).round(3))
# ——————————————————————————————


전체 데이터 분포
0.1
0    13630
1    12753
Name: count, dtype: int64
전체 비율
0.1
0    0.517
1    0.483
Name: count, dtype: float64 

학습 세트 분포
0.1
0    10904
1    10202
Name: count, dtype: int64
학습 세트 비율
0.1
0    0.517
1    0.483
Name: count, dtype: float64 

테스트 세트 분포
0.1
0    2726
1    2551
Name: count, dtype: int64
테스트 세트 비율
0.1
0    0.517
1    0.483
Name: count, dtype: float64


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators':    [100, 300, 500, 800, 1000],
    'max_depth':       [3, 5, 7, 9],
    'learning_rate':   [0.01, 0.05, 0.1, 0.2],
    'subsample':       [0.6, 0.8, 1.0],
    'colsample_bytree':[0.6, 0.8, 1.0],
    'gamma':           [0, 1, 5, 10],
    'reg_alpha':       [0, 0.1, 0.5, 1],
    'reg_lambda':      [1, 1.5, 2, 3]
}

rs = RandomizedSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    param_distributions=param_dist,
    n_iter=50,         # 탐색 횟수
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

rs.fit(X_train, y_train)
print("Best params:", rs.best_params_)
print("CV best score:", rs.best_score_)

# 최적 모델로 테스트 평가
best_model = rs.best_estimator_
y_pred = best_model.predict(X_test)
print("RandomizedSearchCV 적용 후 Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
